Tarefas:
 - Criar um script em python, 
 - Conter dois pandas dataframe populados com os dados pré-definidos 
 - Deverão ser enviados e salvos em um banco de dados MongoDB,
 - Unir essas duas collection de acordo com o campo informado.


O método SAPE 
Problema de Negócio (ou tarefa):
- Criar um script 2 script, envir e salva em mongo unir com mongoDB,desenvolver uma agregação onde unira essas duas collection de acordo com o campo informado, e realizar estes teste.

Saída: ( Produto final )
1. A resposta para a pergunta.
    - crtiar um scrip
2. Formato da entrega
    - Arquivos:
    • Arquivo Python com os pandas dataframes salvando as informações no banco de dados.
    • Agregação criada no MongoDB com os passos descritos neste documento.
    • Collections do MongoDB exportadas.
    • Arquivo World contendo os questionários respondidos.
3. Local da entrega
    - GitHub

Processo ( Passo a Passo ):
1. Passo a passso para construir a tarefa
    - Realizar o calculo da mediana sobre o produto, tipo e cor
2. Definir o formato da entrega ( Os arquivos mencionados)
    
3. Decidir o local de entrega ( github, e link ao recrutador )
    
Entrada ( Fonte de dados ):

1. Fonte de dados
    - foi fornecido no case
2. Ferramentas
    - Python 3.8.0
    - Bibliotecas pandas, pymongo
    - mongoDB Compass
    - Jupyter Notebook (VScode )
 

O raciocínio usado para a solução é o seguinte:

Primeiro, os dataframes pandas são criados.
Em seguida, os dataframes são salvos no MongoDB.
Depois, uma agregação é criada no MongoDB. 
A agregação é usada para combinar as duas coleções "Carros" e "Montadoras" de acordo com o campo "Montadora".
A agregação então agrupa os dados pelo campo "País" e insere as informações sobre o carro dentro de um elemento chamado "Carros".
O resultado da agregação é então impresso na saída.

In [2]:
import pandas as pd
import pymongo
from pymongo import MongoClient
import json


In [2]:
# Dataframe do carro
col_cars = {
    'Carro': ['Onix','Polo','Sandero','Fiesta', 'City'],
    'Cor': ['Prata','Branco', 'Prata', 'Vermelho','Preto'],
    'Montadora': ['Chevrolet','Volkswage','Renolt', 'Ford', 'Honda']
}

# Dataframe da montadora
col_assembler = {
    'Montadora': ['Chevrolet', 'Volkswage', 'Renolt', 'Ford', 'Honda'],
    'Pais': ['EUA', 'Alemanha','Franca','EUA', 'Japao']
}

# 
collection_carros = pd.DataFrame(col_cars)
collection_montadoras = pd.DataFrame(col_assembler)


In [3]:
# Conectando ao MongoDB local
client = MongoClient('mongodb://localhost:27017/')

# Selecionando o banco de dados
db = client['seu_banco_de_dados']

# Salvando os dataframes nas colections
db['Carros'].insert_many(collection_carros.to_dict('records'))
db['Montadoras'].insert_many(collection_montadoras.to_dict('records'))


In [6]:
# Criando a agregação
pipeline = [
    {
        '$lookup': {
            'from': 'Montadoras',
            'localField': 'Montadora',
            'foreignField': 'Montadora',
            'as': 'montadora_info'
        }
    },
    {
        '$unwind': '$montadora_info'
    },
    {
        '$project': {
            '_id': 0,
            'Carro': 1,
            'Cor': 1,
            'Montadora': 1,
            'Pais': '$montadora_info.Pais'
        }
    }
]

# Executando a agregação
aggregation_result = db['Carros'].aggregate(pipeline)

# Imprimindo o resultado
for doc in aggregation_result:
    print(doc)


{'Carro': 'Onix', 'Cor': 'Prata', 'Montadora': 'Chevrolet', 'Pais': 'EUA'}
{'Carro': 'Onix', 'Cor': 'Prata', 'Montadora': 'Chevrolet', 'Pais': 'EUA'}
{'Carro': 'Polo', 'Cor': 'Branco', 'Montadora': 'Volkswage', 'Pais': 'Alemanha'}
{'Carro': 'Polo', 'Cor': 'Branco', 'Montadora': 'Volkswage', 'Pais': 'Alemanha'}
{'Carro': 'Sandero', 'Cor': 'Prata', 'Montadora': 'Renolt', 'Pais': 'Franca'}
{'Carro': 'Sandero', 'Cor': 'Prata', 'Montadora': 'Renolt', 'Pais': 'Franca'}
{'Carro': 'Fiesta', 'Cor': 'Vermelho', 'Montadora': 'Ford', 'Pais': 'EUA'}
{'Carro': 'Fiesta', 'Cor': 'Vermelho', 'Montadora': 'Ford', 'Pais': 'EUA'}
{'Carro': 'City', 'Cor': 'Preto', 'Montadora': 'Honda', 'Pais': 'Japao'}
{'Carro': 'City', 'Cor': 'Preto', 'Montadora': 'Honda', 'Pais': 'Japao'}
{'Carro': 'Onix', 'Cor': 'Prata', 'Montadora': 'Chevrolet', 'Pais': 'EUA'}
{'Carro': 'Onix', 'Cor': 'Prata', 'Montadora': 'Chevrolet', 'Pais': 'EUA'}
{'Carro': 'Polo', 'Cor': 'Branco', 'Montadora': 'Volkswage', 'Pais': 'Alemanha'}
{'Car

In [14]:


# # Convertendo a agregação em uma string JSON
# aggregation_json = json.dumps(list(aggregation_result))

# # Salvando a agregação em um arquivo .js
# with open('aggregation.js', 'w') as f:
#     f.write(aggregation_json)


In [7]:


# Executando a agregação
aggregation_result = db['Carros'].aggregate(pipeline)

# Convertendo a agregação em uma lista
result_list = list(aggregation_result)

# Convertendo a agregação em uma string JSON
aggregation_json = json.dumps(result_list)

# Salvando a agregação em um arquivo .js
with open('aggregation.js', 'w') as f:
    f.write(aggregation_json)
